In [38]:
import os
import time
import random
import pandas as pd
from gtts import gTTS
from psychopy import sound, core
import psychtoolbox as ptb

def administer_sentences(noun_list, adj_list, verb_list, num_sentences=1):
    
    administered_sentences = []

    sentence_count = 0

    while sentence_count < num_sentences:
    
        for i in range(num_sentences):

            # Generate the random sentence from the lists of words
            noun = random.choice(noun_list)
            adj = random.choice(adj_list)
            verb = random.choice(verb_list)
            sentence = f"{noun} is {adj} and {verb}"

            # Capture timestamp of administered sentence
            timestamp = time.time()

            # Initialize gTTS and play the sentence audio
            tts = gTTS(text=sentence, lang="en")
            tts.save("temp_sentence.mp3")
            sentence_sound = sound.Sound("temp_sentence.mp3")
            now = ptb.GetSecs()
            sound.Sound.play(sentence_sound, when=now)

            # Wait for 5 seconds before looping again
            core.wait(5)

            # Increment the sentence count
            sentence_count += 1

            # Delete the intermediate mp3 file
            if os.path.exists("temp_sentence.mp3"):
                os.remove("temp_sentence.mp3")

            # Save the sentence, timestamp tuple to administered sentences list
            administered_sentences.append((sentence, timestamp))
    
    return administered_sentences

def administer_word(word_list):

    # Initialize word to be played by gTTS
    word = random.choice(word_list)

    # Get timestamp of when word is played
    timestamp = time.time()
    
    # Initialize gTTS and play the sentence audio
    tts = gTTS(text=word, lang="en")
    tts.save("temp_word.mp3")
    word_sound = sound.Sound("temp_word.mp3")
    now = ptb.GetSecs()
    sound.Sound.play(word_sound, when=now)

    # Wait for 5 seconds before next stimulus
    core.wait(5)

    # Delete intermediate mp3 file
    if os.path.exists("temp_word.mp3"):
        os.remove("temp_word.mp3")

    return word, timestamp

def administer_beep(frequency=1000, duration=0.5):

    # Get timestamp of when beep will play
    timestamp = time.time()
    
    # Initialize sound
    beep_sound = sound.Sound(value=frequency, secs=duration)
    now = ptb.GetSecs()
    sound.Sound.play(beep_sound, when=now)
    core.wait(beep_sound.getDuration())

    return "BEEP", timestamp

In [39]:
def get_random_stimulus_order():
    stimuli = ["sentence", "word", "beep"]
    random.shuffle(stimuli)
    return stimuli


In [40]:
def generate_and_play_stimuli(num_sentences=4, patient_id="patient0"):
    noun_list_path = os.path.join("word_lists/noun_list.txt")
    adj_list_path = os.path.join("word_lists/adj_list.txt")
    verb_list_path = os.path.join("word_lists/verb_list.txt")
    word_list_path = os.path.join("word_lists/word_list.txt")

    with open(noun_list_path, "r") as noun_file:
        noun_list = [word.strip() for line in noun_file for word in line.split(',')]

    with open(adj_list_path, "r") as adj_file:
        adj_list = [word.strip() for line in adj_file for word in line.split(',')]

    with open(verb_list_path, "r") as verb_file:
        verb_list = [word.strip() for line in verb_file for word in line.split(',')]

    with open(word_list_path, "r") as word_file:
        word_list = [word.strip() for line in word_file for word in line.split(',')]

    current_date = time.strftime("%Y-%m-%d")

    if os.path.exists('patient_df.csv'):
        patient_df = pd.read_csv('patient_df.csv')
    else:
        patient_df = pd.DataFrame(columns=['patient_id', 'date', 'stimulus', 'timestamp', 'order'])

    administered_stimuli = []

    if ((patient_df['patient_id'] == patient_id) & (patient_df['date'] == current_date)).any():
        print("Patient has already been administered stimulus protocol today")
        return
    else:
        stimulus_order = get_random_stimulus_order()

        for stimulus in stimulus_order:
            if stimulus == "sentence":
                result_list = administer_sentences(noun_list, adj_list, verb_list, num_sentences)
                for sentence, timestamp in result_list:
                    administered_stimuli.append({
                        'patient_id': patient_id,
                        'date': current_date,
                        'stimulus': sentence,
                        'timestamp': timestamp,
                        'order': ", ".join(stimulus_order)
                    })
            elif stimulus == "word":
                word, timestamp = administer_word(word_list)
                administered_stimuli.append({
                    'patient_id': patient_id,
                    'date': current_date,
                    'stimulus': word,
                    'timestamp': timestamp,
                    'order': ", ".join(stimulus_order)
                })
            elif stimulus == "beep":
                beep, timestamp = administer_beep()
                administered_stimuli.append({
                    'patient_id': patient_id,
                    'date': current_date,
                    'stimulus': beep,
                    'timestamp': timestamp,
                    'order': ", ".join(stimulus_order)
                })

            core.wait(5)

    administered_stimuli_df = pd.concat([patient_df, pd.DataFrame(administered_stimuli)], ignore_index=True)
    administered_stimuli_df.to_csv("patient_df.csv", index=False)

# Example usage:
generate_and_play_stimuli(num_sentences=4, patient_id="patient2")
